### Import chipwhisperer and set the scope

The scope can be left as is, because we have only one device connected. In cases where there are multiple divices connected we can use `scope(sn=DEVICE_SERIAL_NUMBER)` to create scope for a specific device. 

In [ ]:
import chipwhisperer as cw
scope = cw.scope()

Note to self:
When the scope is created, the device's green LED light starts blinking (on top of the blue LED that probably indicates that the device is connected physically).

We also set the scope to a default setup which means:

- Sets the scope gain to 45dB
- Sets the scope to capture 5000 samples
- Sets the scope offset to 0 (aka it will begin capturing as soon as it is triggered)
- Sets the scope trigger to rising edge
- Outputs a 7.37MHz clock to the target on HS2
- Clocks the scope ADC at 4*7.37MHz. Note that this is synchronous to the target clock on HS2
- Assigns GPIO1 as serial RX
- Assigns GPIO2 as serial TX

In [ ]:
scope.default_setup()

### Connecting to target

Now we can connect to the target (probably meaning the target board). 

In [ ]:
target = cw.target(scope, cw.targets.SimpleSerial2) #cw.targets.SimpleSerial can be omitted
#MY CHANGES - changed target to SimpleSerial2 - to be able to send_cmd

In [ ]:
%%bash
cd version_01/
make PLATFORM='CWLITEARM' CRYPTO_TARGET=NONE

### Flashing The Compiled .hex Onto Target

In [ ]:
cw.program_target(scope, cw.programmers.STM32FProgrammer, "version_01/simpleserial-target-CWLITEARM.hex")

### Capturing a trace

In [ ]:
# arm the scope
scope.arm()

# Set the maximum number of points in a trace
scope.adc.fifo_fill_mode = "normal"
scope.adc.samples = 24431
scope.adc.decimate = 2


# flush the UART buffer
target.flush()

data = bytearray([0x42] * 16)
target.send_cmd('p', 0x80, data)

ret = scope.capture()
trace = scope.get_last_trace()

returned_data = target.read_cmd('r')
print(returned_data)
ack = target.read_cmd('e')
print(ack)

In [ ]:
print(len(trace))

In [ ]:
import matplotlib.pyplot as plt
# Plot the trace
plt.plot(trace, linewidth=0.8)

plt.savefig('figure.svg', format='svg')
plt.show()

In [ ]:
print(returned_data)
byte_arr = bytearray(returned_data)
print(byte_arr)


### Trying Segmented Capture

In [ ]:
scope.arm()
scope.adc.fifo_fill_mode = "segment"
scope.adc.samples = 21000
# flush the UART buffer
target.flush()

data = bytearray([0x42] * 16)
target.send_cmd('p', 0x80, data)
traces = []

ret = scope.capture_segmented()
trace = scope.get_last_trace_segmented()

total_trace = []
print(len(trace))
for t in trace:
    print(len(t))
    for p in t:
        total_trace.append(p)
        
ret = scope.capture_segmented()
trace = scope.get_last_trace_segmented()

print(len(trace))
for t in trace:
    print(len(t))
    for p in t:
        total_trace.append(p)
    
#while captured_points < total_points:
#        
#    ret = scope.capture_segmented()
#    trace = scope.get_last_trace_segmented()
#    
#    traces.append(trace)
#    print(len(trace))
#    captured_points += len(trace)

In [ ]:
plt.plot(total_trace, linewidth=0.8)

plt.savefig('figure_segmented.svg', format='svg')
plt.show()

### Disconnecting the target and the scope

In [ ]:
target.dis()

In [ ]:
scope.dis()